# Introducción

En este proyecto, se aborda el problema de la evaluación de similitud semántica entre pares de oraciones. Este es un desafío fundamental en el procesamiento del lenguaje natural (NLP), con aplicaciones en tareas como recuperación de información, detección de plagio, sistemas de recomendación y más.

Este trabajo forma parte de la materia **Deep Learning para NLP**, donde se exploran técnicas avanzadas para resolver problemas complejos en el ámbito del lenguaje natural.

Para resolver este problema, se implementan y comparan tres enfoques basados en modelos de lenguaje preentrenados (BERT y variantes optimizadas para similitud semántica). Los modelos utilizados son:

1. **BERT + Regresión**: Un modelo basado en BERT con una capa de regresión para predecir la similitud.
2. **Siamese BERT**: Un modelo siamés que compara las representaciones de dos oraciones.
3. **Cross-Attention Model**: Un modelo que utiliza atención cruzada para capturar interacciones entre las oraciones.

El objetivo es entrenar y evaluar estos modelos en el conjunto de datos STS Benchmark, normalizando las etiquetas a un rango de 0 a 1. Finalmente, se validan los modelos con ejemplos de prueba para analizar su desempeño en diferentes niveles de similitud semántica.

In [48]:
import os
import torch
import numpy as np
import random
import evaluate
import pandas as pd
import textwrap
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from datasets import load_dataset
from torch.optim import AdamW
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModel, get_scheduler
from transformers import DataCollatorWithPadding
import psutil
import ipywidgets as widgets
from IPython.display import display

Configuracion para la Reproducibilidad

In [49]:
# Seed para asegurar reproducibilidad sin importar el dispositivo en el que se ejecute
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

set_seed()

### Carga y Preprocesamiento del Dataset

In [50]:
# Cargar el dataset
dataset = load_dataset("mteb/stsbenchmark-sts")

# Normalizar las etiquetas (de 0-5 a un rango de 0-1)
def normalize_labels(examples):
    examples["score"] = [s / 5.0 for s in examples["score"]]
    return examples

# Aplicar normalización
dataset = dataset.map(normalize_labels, batched=True)


In [51]:
print(dataset)
print(f"Memoria usada dataset: {psutil.virtual_memory().percent}%")

DatasetDict({
    train: Dataset({
        features: ['split', 'genre', 'dataset', 'year', 'sid', 'score', 'sentence1', 'sentence2'],
        num_rows: 5749
    })
    validation: Dataset({
        features: ['split', 'genre', 'dataset', 'year', 'sid', 'score', 'sentence1', 'sentence2'],
        num_rows: 1500
    })
    test: Dataset({
        features: ['split', 'genre', 'dataset', 'year', 'sid', 'score', 'sentence1', 'sentence2'],
        num_rows: 1379
    })
})
Memoria usada dataset: 60.3%


Tokenizacion del Texto

In [52]:
#Creamos un tokenizador para convertir las frases en IDs de tokens que el modelo pueda procesar. Usare BERT o un modelo optimizado para similitud semántica
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/paraphrase-MiniLM-L6-v2")

#Definiendo la funcion de tokenizacion
def tokenize_function(examples):
    #Tokenizando sentence1 y sentence2 por separado
    #Esto es necesario porque el modelo espera que las entradas sean pares de oraciones
    #y no una sola oración.
    encodings1 = tokenizer(
        examples['sentence1'],
        padding =False,
        truncation=True,
        max_length=128,
        return_tensors=None # Lo dejo como lista para el dataset
    )
    encodings2 = tokenizer(
        examples['sentence2'],
        padding=False,
        truncation=True,
        max_length=128,
        return_tensors=None # Lo dejo como lista para el dataset
    )
    return{
        'input_ids1': encodings1['input_ids'],
        'attention_mask1': encodings1['attention_mask'],
        'input_ids2': encodings2['input_ids'],
        'attention_mask2': encodings2['attention_mask'],
        'score': examples['score'] # Mantengo la etiqueta original
    }
    
#Tokenizando el dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Probando 123...

In [53]:
print(tokenized_datasets["train"][0]["input_ids1"])
print(tokenized_datasets["train"][0]["input_ids2"])
print(len(tokenized_datasets["train"][0]["input_ids1"]), len(tokenized_datasets["train"][0]["input_ids2"]))
print(f"Memoria usada tokenizacion: {psutil.virtual_memory().percent}%")

[101, 1037, 4946, 2003, 2635, 2125, 1012, 102]
[101, 2019, 2250, 4946, 2003, 2635, 2125, 1012, 102]
8 9
Memoria usada tokenizacion: 60.3%


Creando el Dataset para Pytorch

In [54]:
#Creo una clase para formatear los datos de entrada correctamente para PyTorch
class STSDataset(Dataset):
    def __init__(self, data):        
        self.input_ids1 = data["input_ids1"]
        self.attention_mask1 = data["attention_mask1"]
        self.input_ids2 = data["input_ids2"]
        self.attention_mask2 = data["attention_mask2"]
        self.scores = data["score"]  # Los scores son las etiquetas (labels)

        
    def __len__(self):
        return len(self.input_ids1)
    
    def __getitem__(self, idx):
        return{
            'input_ids1': self.input_ids1[idx],
            'attention_mask1': self.attention_mask1[idx],
            'input_ids2': self.input_ids2[idx],
            'attention_mask2': self.attention_mask2[idx],
            'labels': self.scores[idx],
        }
#Creando el datase de entrenamiento y validacion
train_dataset = STSDataset(tokenized_datasets["train"])
val_dataset = STSDataset(tokenized_datasets["validation"])
test_dataset = STSDataset(tokenized_datasets["test"])

# Creando la Funcion para Fine-Tuning en los Modelos

In [55]:
# Se crea un Widget para diferentes parametros de configuracion e identificar la mejor opcion.
learning_rate = widgets.FloatText(value=1e-5, description="learning_rate:")
freeze_layers = widgets.IntText(value=4, description="freeze_layers:")
epochs = widgets.IntText(value=20, description="epochs:")
scheduler_type = widgets.Text(value='linear', description="scheduler_type:")

display(learning_rate, freeze_layers, epochs, scheduler_type)


lr = learning_rate.value
fl = freeze_layers.value
ep = epochs.value
st = scheduler_type.value


FloatText(value=1e-05, description='learning_rate:')

IntText(value=4, description='freeze_layers:')

IntText(value=20, description='epochs:')

Text(value='linear', description='scheduler_type:')

In [56]:
# Función para congelar capas
def freeze_bert_layers(model, num_frozen_layers=fl):
    """
    Congela las primeras `num_frozen_layers` capas del modelo BERT.
    """
    for layer in model.bert.encoder.layer[:num_frozen_layers]:
        for param in layer.parameters():
            param.requires_grad = False

    # Congelamos las embeddings iniciales también
    for param in model.bert.embeddings.parameters():
        param.requires_grad = False


### Definiendo los Modelos

Modelo 1 : Siamese BERT

In [57]:
#Definiendo el primer modelo a utilizar, en este caso estaremos utilizando el modelo de Sentence Transformers "all-MiniLM-L6-v2"
#Creo un modelo basado en BERT con un head de regresión
class SentenceSimilarityModelOne(torch.nn.Module):
    def __init__(self, model_name="sentence-transformers/all-MiniLM-L6-v2", freeze_layers=fl):
        super().__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        
        # Congelar capas según parámetro freeze_layers
        for i, param in enumerate(self.bert.encoder.layer):
            if i < (len(self.bert.encoder.layer) - freeze_layers):
                for p in param.parameters():
                    p.requires_grad = False
                    
        self.regressor = torch.nn.Sequential(
            torch.nn.Linear(384, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 1)
        )
    
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        cls_embeddings = outputs.last_hidden_state[:, 0, :]
        similarity = self.regressor(cls_embeddings)
        return similarity.squeeze()

Modelo 2: Siamese BERT

In [58]:
class SiameseBERT(torch.nn.Module):
    def __init__(self, model_name="sentence-transformers/all-MiniLM-L6-v2", freeze_layers=fl):
        super().__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        
        # Congelar capas según parámetro freeze_layers
        for i, param in enumerate(self.bert.encoder.layer):
            if i < (len(self.bert.encoder.layer) - freeze_layers):
                for p in param.parameters():
                    p.requires_grad = False
           
        self.regressor = torch.nn.Sequential(
            torch.nn.Linear(384 * 3, 256),
            torch.nn.ReLU(),
            torch.nn.Linear(256, 1)
        )

    def forward(self, input_ids1, attention_mask1, input_ids2, attention_mask2):
        output1 = self.bert(input_ids1, attention_mask=attention_mask1).last_hidden_state[:, 0, :]
        output2 = self.bert(input_ids2, attention_mask=attention_mask2).last_hidden_state[:, 0, :]

        diff = torch.abs(output1 - output2)
        mult = output1 * output2
        combined = torch.cat([diff, mult, output1], dim=1)

        return self.regressor(combined).squeeze()


Modelo 3: Cross-Attention Model

In [59]:
class CrossAttentionModel(torch.nn.Module):
    def __init__(self, model_name="sentence-transformers/all-MiniLM-L6-v2", freeze_layers=fl):
        super().__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        for i, param in enumerate(self.bert.encoder.layer):
            if i < (len(self.bert.encoder.layer) - freeze_layers):
                for p in param.parameters():
                    p.requires_grad = False
        self.attention = torch.nn.MultiheadAttention(embed_dim=384, num_heads=8)
        self.regressor = torch.nn.Sequential(
            torch.nn.Linear(384 * 2, 256),
            torch.nn.ReLU(),
            torch.nn.Linear(256, 1)
        )

    def forward(self, input_ids1, attention_mask1, input_ids2, attention_mask2):
        output1 = self.bert(input_ids1, attention_mask=attention_mask1).last_hidden_state
        output2 = self.bert(input_ids2, attention_mask=attention_mask2).last_hidden_state
        print(f"output1 shape: {output1.shape}")
        print(f"output2 shape: {output2.shape}")
        output1 = output1.transpose(0, 1)  # [seq_len, batch_size, hidden_size]
        output2 = output2.transpose(0, 1)  # [seq_len, batch_size, hidden_size]
        attn_output, _ = self.attention(output1, output2, output2)
        attn_output = attn_output.transpose(0, 1)  # [batch_size, seq_len, hidden_size]
        print(f"attn_output shape: {attn_output.shape}")
        combined = torch.cat([output1.transpose(0, 1)[:, 0, :], attn_output[:, 0, :]], dim=1)
        return self.regressor(combined).squeeze()

### Configuracion el Entrenamiento

In [60]:
class CustomDataCollatorWithPadding:
    def __init__(self, tokenizer, max_length=128):
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.padding_collator = DataCollatorWithPadding(tokenizer, max_length=max_length)
    
    def __call__(self, examples):
        features1 = [
            {'input_ids': ex['input_ids1'], 'attention_mask': ex['attention_mask1']}
            for ex in examples
        ]
        features2 = [
            {'input_ids': ex['input_ids2'], 'attention_mask': ex['attention_mask2']}
            for ex in examples
        ]
        labels = [ex['labels'] for ex in examples]
        batch1 = self.padding_collator(features1)
        batch2 = self.padding_collator(features2)
        batch = {
            'input_ids1': batch1['input_ids'],
            'attention_mask1': batch1['attention_mask'],
            'input_ids2': batch2['input_ids'],
            'attention_mask2': batch2['attention_mask'],
            'labels': torch.tensor(labels, dtype=torch.float32)
        }
        return batch

data_collator = CustomDataCollatorWithPadding(tokenizer, max_length=128)

In [61]:
# Definiendo el optimizador y el scheduler
train_loader = DataLoader(
    train_dataset, 
    batch_size=8, 
    shuffle=True,
    collate_fn=data_collator,  # Usar el collator personalizado
    )
for batch in train_loader:
    print(f"input_ids1 shape: {batch['input_ids1'].shape}")
    print(f"attention_mask1 shape: {batch['attention_mask1'].shape}")
    print(f"input_ids2 shape: {batch['input_ids2'].shape}")
    print(f"attention_mask2 shape: {batch['attention_mask2'].shape}")
    break
val_loader = DataLoader(
    val_dataset, 
    batch_size=8, 
    shuffle=False,
    collate_fn=data_collator  # Usar el collator personalizado
    )
test_loader = DataLoader(
    test_dataset,
    batch_size=8,
    shuffle=False,
    collate_fn=data_collator # Uso el collator personalizado

)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
model1 = SentenceSimilarityModelOne().to(device)
model2 = SiameseBERT().to(device)
model3 = CrossAttentionModel().to(device)


for model in [model1, model2, model3]:
    optimizer = AdamW(model.parameters(), lr=lr)
    loss_fn = torch.nn.MSELoss()

input_ids1 shape: torch.Size([8, 23])
attention_mask1 shape: torch.Size([8, 23])
input_ids2 shape: torch.Size([8, 18])
attention_mask2 shape: torch.Size([8, 18])
Using device: cuda


In [62]:
for batch in train_loader:
    print("Shape de input_ids1:", batch["input_ids1"].shape)
    print("Shape de input_ids2:", batch["input_ids2"].shape)
    print("Ejemplo de input_ids1:", batch["input_ids1"][0])
    print("Ejemplo de input_ids2:", batch["input_ids2"][0])
    break

Shape de input_ids1: torch.Size([8, 27])
Shape de input_ids2: torch.Size([8, 33])
Ejemplo de input_ids1: tensor([  101,  7387,  2118,  3742,  4457,  2011, 28932,   102,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0])
Ejemplo de input_ids2: tensor([  101,  7387,  1024,  2118, 18897,  1999,  2167,   102,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0])


Validando el Fine - Tuning de los Modelos

In [63]:
def print_trainable_params(model):
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total_params = sum(p.numel() for p in model.parameters())
    print(f"Total Parámetros: {total_params:,}, Entrenables: {trainable_params:,}")

print("\nBERT + Regresión:")
print_trainable_params(model1)

print("\nSiamese BERT:")
print_trainable_params(model2)

print("\nCross-Attention:")
print_trainable_params(model3)



BERT + Regresión:
Total Parámetros: 22,762,625, Entrenables: 19,213,697

Siamese BERT:
Total Parámetros: 23,008,641, Entrenables: 19,459,713

Cross-Attention:
Total Parámetros: 23,501,697, Entrenables: 19,952,769


Monitoreo de los recursos

In [64]:
print(f"Memoria usada antes de los procesos: {psutil.virtual_memory().percent}%")

Memoria usada antes de los procesos: 60.5%


### Evaluación del Modelo

In [65]:
# Función de evaluación
def evaluate_model(model, val_loader, model_type="bert"):
    model.eval()
    preds, labels = [], []

    with torch.no_grad():
        for batch in val_loader:
            if model_type == "bert":
                inputs = batch["input_ids1"].to(device)
                mask = batch["attention_mask1"].to(device)
                lbls = batch["labels"].to(device)
                outputs = model(inputs, mask)

            elif model_type in ["siamese", "cross"]:
                inputs1 = batch["input_ids1"].to(device)
                mask1 = batch["attention_mask1"].to(device)
                inputs2 = batch["input_ids2"].to(device)
                mask2 = batch["attention_mask2"].to(device)
                lbls = batch["labels"].to(device)

                outputs = model(inputs1, mask1, inputs2, mask2)

            preds.extend(outputs.cpu().numpy())
            labels.extend(lbls.cpu().numpy())

    correlation, _ = pearsonr(preds, labels)
    print(f"{model_type.upper()} - Pearson Correlation: {correlation:.4f}")
    return correlation

In [66]:
# def weighted_mse_loss(output, target):
#     weights = torch.where(target < 1,2.0,1.0) # para similitudes muy bajas, se penaliza más
#     return torch.mean(weights * (output - target) ** 2)

# Función para entrenar modelo con diferentes configuraciones
def train_model(model, train_loader, val_loader, model_type="bert", epochs=ep, output_dir="./checkpoints"):
    model.train()
    optimizer = AdamW(model.parameters(), lr=lr) #
    loss_fn = torch.nn.MSELoss()
    scheduler = get_scheduler(st, optimizer=optimizer, num_warmup_steps=0, num_training_steps=len(train_loader) * epochs)
    history = []
    best_pearson = -float("inf")
    best_model_path = os.path.join(output_dir, f"best_model_{model_type}.pt")
    
    os.makedirs(output_dir, exist_ok=True)
    print(f"Memoria usada 1: {psutil.virtual_memory().percent}%")

    for epoch in range(epochs):
        total_loss = 0
        model.train()
        
        for batch in train_loader:
            optimizer.zero_grad()
            #print(f"Memoria usada 2: {psutil.virtual_memory().percent}%")
            if model_type == "bert":
                inputs = batch["input_ids1"].to(device)
                mask = batch["attention_mask1"].to(device)
                labels = batch["labels"].to(device)
                outputs = model(inputs, mask)

            elif model_type in ["siamese", "cross"]:
                inputs1 = batch["input_ids1"].to(device)
                mask1 = batch["attention_mask1"].to(device)
                inputs2 = batch["input_ids2"].to(device)
                mask2 = batch["attention_mask2"].to(device)
                labels = batch["labels"].to(device)
                outputs = model(inputs1, mask1, inputs2, mask2)

            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()
            scheduler.step()
            total_loss += loss.item()
            
            # Libero los tensores explícitamente para que no ocupen memoria
            #del inputs1, mask1, inputs2, mask2, labels, outputs, loss
            torch.cuda.empty_cache()  # Para cuando uso la GPU
            #print(f"Memoria usada 3: {psutil.virtual_memory().percent}%")
        avg_train_loss = total_loss / len(train_loader)
         # Evaluo después de cada época
        pearson_corr = evaluate_model(model, val_loader, model_type)
        history.append((avg_train_loss, pearson_corr))
        print(f"Memoria usada 1: {psutil.virtual_memory().percent}%")
        print(f"{model_type.upper()} - Epoch {epoch+1}/{epochs} - Loss: {avg_train_loss:.4f}")
        #print(f"Memoria usada: {psutil.virtual_memory().percent}%")
        # Guardar el mejor modelo basado en la correlación de Pearson
        if pearson_corr > best_pearson:
            best_pearson = pearson_corr
            torch.save(model.state_dict(), best_model_path)
            print(f"Mejor modelo guardado en {best_model_path} con correlación de Pearson: {best_pearson:.4f}")
            
    # Cargo el mejor modelo despues de entrenar
    model.load_state_dict(torch.load(best_model_path))
    print(f"{model_type.upper()} - Entrenamiento finalizado. Mejor modelo cargado: {best_pearson:.4f}")
    
    return history, best_pearson # Devuelvo el historial de pérdidas y correlaciones mas el mejor modelo

def evaluate_on_test(model, test_loader, model_type="bert"):
    person_corr = evaluate_model(model, test_loader, model_type)
    print(f"{model_type.upper()} - Evaluación en test - Pearson Correlation: {person_corr:.4f}")
    return person_corr

In [67]:
def infer_similarity(model, tokenizer, sentence1, sentence2):
    model.eval()
    with torch.no_grad():
        encoded_input = tokenizer(
            sentence1,
            sentence2,
            padding="max_length",
            truncation=True,
            max_length=128,
            return_tensors="pt"
        ).to(device)

        if isinstance(model, SiameseBERT) or isinstance(model, CrossAttentionModel):
            input_ids1 = encoded_input["input_ids"]
            attention_mask1 = encoded_input["attention_mask"]
            input_ids2 = encoded_input["input_ids"]
            attention_mask2 = encoded_input["attention_mask"]

            similarity = model(input_ids1, attention_mask1, input_ids2, attention_mask2)
        else:
            similarity = model(encoded_input["input_ids"], encoded_input["attention_mask"])

    return similarity.item() * 5  # Volvemos a la escala de 0 a 5


In [68]:
#Entrenando y evaluando los modelos
models = {
    #"BERT + Regresion": (model1, "bert"),
    "Siamese BERT": (model2, "siamese"),
    "Cross-Attention": (model3, "cross")
}

test_results = {}
for model_name, (model, model_type) in models.items():
    print(f"\nEntrenando {model_name}...")
    history, best_pearson = train_model(model, train_loader, val_loader, model_type=model_type, epochs=ep)
    test_results[model_name] = best_pearson
    print(f"Correlación de Pearson en validación para {model_name}: {best_pearson:.4f}")
    
    
for model_name, (model, model_type) in models.items():
    print(f"\nEntrenando {model_name} en el conjunto de test...")
    test_pearson = evaluate_on_test(model, test_loader, model_type)
    test_results[model_name] = test_pearson
    print(f"Correlación de Pearson en test para {model_name}: {test_pearson:.4f}")


Entrenando Siamese BERT...
Memoria usada 1: 62.2%
SIAMESE - Pearson Correlation: 0.8222
Memoria usada 1: 62.0%
SIAMESE - Epoch 1/20 - Loss: 0.0632
Mejor modelo guardado en ./checkpoints\best_model_siamese.pt con correlación de Pearson: 0.8222
SIAMESE - Pearson Correlation: 0.8534
Memoria usada 1: 62.1%
SIAMESE - Epoch 2/20 - Loss: 0.0334
Mejor modelo guardado en ./checkpoints\best_model_siamese.pt con correlación de Pearson: 0.8534
SIAMESE - Pearson Correlation: 0.8623
Memoria usada 1: 61.9%
SIAMESE - Epoch 3/20 - Loss: 0.0274
Mejor modelo guardado en ./checkpoints\best_model_siamese.pt con correlación de Pearson: 0.8623
SIAMESE - Pearson Correlation: 0.8661
Memoria usada 1: 61.7%
SIAMESE - Epoch 4/20 - Loss: 0.0241
Mejor modelo guardado en ./checkpoints\best_model_siamese.pt con correlación de Pearson: 0.8661
SIAMESE - Pearson Correlation: 0.8690
Memoria usada 1: 61.7%
SIAMESE - Epoch 5/20 - Loss: 0.0224
Mejor modelo guardado en ./checkpoints\best_model_siamese.pt con correlación de 

In [69]:
#Entrenando y evaluando los modelos
models = {
    "BERT + Regresion": (model1, "bert"),
    # "Siamese BERT": (model2, "siamese"),
    # "Cross-Attention": (model3, "cross")
}

test_results = {}
for model_name, (model, model_type) in models.items():
    print(f"\nEntrenando {model_name}...")
    history, best_pearson = train_model(model, train_loader, val_loader, model_type=model_type, epochs=ep)
    test_results[model_name] = best_pearson
    print(f"Correlación de Pearson en validación para {model_name}: {best_pearson:.4f}")
    
    
for model_name, (model, model_type) in models.items():
    print(f"\nEntrenando {model_name} en el conjunto de test...")
    test_pearson = evaluate_on_test(model, test_loader, model_type)
    test_results[model_name] = test_pearson
    print(f"Correlación de Pearson en test para {model_name}: {test_pearson:.4f}")


Entrenando BERT + Regresion...
Memoria usada 1: 72.5%
BERT - Pearson Correlation: 0.2087
Memoria usada 1: 72.3%
BERT - Epoch 1/20 - Loss: 0.0883
Mejor modelo guardado en ./checkpoints\best_model_bert.pt con correlación de Pearson: 0.2087
BERT - Pearson Correlation: 0.2111
Memoria usada 1: 72.5%
BERT - Epoch 2/20 - Loss: 0.0774
Mejor modelo guardado en ./checkpoints\best_model_bert.pt con correlación de Pearson: 0.2111
BERT - Pearson Correlation: 0.2226
Memoria usada 1: 72.5%
BERT - Epoch 3/20 - Loss: 0.0739
Mejor modelo guardado en ./checkpoints\best_model_bert.pt con correlación de Pearson: 0.2226
BERT - Pearson Correlation: 0.2248
Memoria usada 1: 72.7%
BERT - Epoch 4/20 - Loss: 0.0703
Mejor modelo guardado en ./checkpoints\best_model_bert.pt con correlación de Pearson: 0.2248
BERT - Pearson Correlation: 0.1854
Memoria usada 1: 72.5%
BERT - Epoch 5/20 - Loss: 0.0656
BERT - Pearson Correlation: 0.2101
Memoria usada 1: 72.3%
BERT - Epoch 6/20 - Loss: 0.0621
BERT - Pearson Correlation:

In [70]:
models = {
    "BERT + Regresion": (model1, "bert"),
    "Siamese BERT": (model2, "siamese"),
    "Cross-Attention": (model3, "cross")
}

Validando los 6 ejemplo contra los modelos entrenados

In [71]:
models = {
    "BERT + Regresion": (model1, "bert"),
    "Siamese BERT": (model2, "siamese"),
    "Cross-Attention": (model3, "cross")
}

test_sentences = [
    ("I love eating apples", "The capital of France is Paris"),  # Similitud 0 (Completamente distintas)
    ("I have a black cat", "My pet is a dog"),  # Similitud 1 (Diferentes pero relacionadas con mascotas)
    ("He plays soccer on weekends", "She enjoys playing tennis on Sundays"),  # Similitud 2 (Acciones similares, pero no iguales)
    ("The sun is shining in the sky", "It is a bright and sunny day"),  # Similitud 3 (Mismo contexto, expresado diferente)
    ("The smartphone has a large screen", "This phone features a big display"),  # Similitud 4 (Misma idea, palabras diferentes)
    ("The dog is barking loudly", "The dog is barking loudly"),  # Similitud 5 (Frases idénticas)
]

# Evaluando los modelos en ejemplos de prueba
for model_name, (model, model_type) in models.items():
    print(f"\nEvaluando {model_name} en ejemplos de prueba:")
    for i, (s1, s2) in enumerate(test_sentences):
        score = infer_similarity(model, tokenizer, s1, s2)
        print(f"Ejemplo {i+1}: {s1} | {s2} → Score: {score:.2f} --> {round(score)}")
        


Evaluando BERT + Regresion en ejemplos de prueba:
Ejemplo 1: I love eating apples | The capital of France is Paris → Score: 3.05 --> 3
Ejemplo 2: I have a black cat | My pet is a dog → Score: 2.82 --> 3
Ejemplo 3: He plays soccer on weekends | She enjoys playing tennis on Sundays → Score: 2.37 --> 2
Ejemplo 4: The sun is shining in the sky | It is a bright and sunny day → Score: 3.12 --> 3
Ejemplo 5: The smartphone has a large screen | This phone features a big display → Score: 3.24 --> 3
Ejemplo 6: The dog is barking loudly | The dog is barking loudly → Score: 3.42 --> 3

Evaluando Siamese BERT en ejemplos de prueba:
Ejemplo 1: I love eating apples | The capital of France is Paris → Score: 6.40 --> 6
Ejemplo 2: I have a black cat | My pet is a dog → Score: 6.47 --> 6
Ejemplo 3: He plays soccer on weekends | She enjoys playing tennis on Sundays → Score: 5.94 --> 6
Ejemplo 4: The sun is shining in the sky | It is a bright and sunny day → Score: 6.45 --> 6
Ejemplo 5: The smartphone has 

In [72]:
import torch
print(f"CUDA disponible: {torch.cuda.is_available()}")
print(f"Nombre de la GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'No GPU'}")
print(f"Versión de PyTorch: {torch.__version__}")
print(f"Versión de CUDA: {torch.version.cuda}")


CUDA disponible: True
Nombre de la GPU: NVIDIA GeForce RTX 3070 Ti Laptop GPU
Versión de PyTorch: 2.6.0+cu118
Versión de CUDA: 11.8
